In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#This is my practice Notebook for Regression model! Full Credit for Cheol Min Kwon's book 'Perfect ML guide with Python'

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

house_df_org = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
house_df = house_df_org.copy()
house_df.head()

In [ ]:
print('데이터 세트의 shape:',house_df.shape)
print('|n전체 피처의 type\n',house_df.dtypes.value_counts())
isnull_series = house_df.isnull().sum()
print('\nNull 칼럼과 그 건수:\n',isnull_series[isnull_series>0].sort_values(ascending=False))

In [ ]:
plt.title('Original Sale Price Histogram')
sns.distplot(house_df['SalePrice'])

In [ ]:
plt.title('Log Transformed Sale Price Histogram')
log_SalePrice = np.log1p(house_df['SalePrice'])
sns.distplot(log_SalePrice)

In [ ]:
original_SalePrice = house_df['SalePrice']
house_df['SalePrice'] = np.log1p(house_df['SalePrice'])

house_df.drop(['Id','PoolQC','MiscFeature','Alley','Fence','FireplaceQu'],axis=1,inplace=True)

house_df.fillna(house_df.mean(),inplace=True)

In [ ]:
null_column_count = house_df.isnull().sum()[house_df.isnull().sum()>0]
print(house_df.dtypes[null_column_count.index])

In [ ]:
print('Before One Hot Encoding:',house_df.shape)
house_df_ohe = pd.get_dummies(house_df)
print('After Ohe Hot Encoding:',house_df_ohe.shape)

In [ ]:
def get_rmse(model):
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test,pred)
    rmse = np.sqrt(mse)
    print(model.__class__.__name__,'로그 변환된 RMSE:',np.round(rmse,3))
    return rmse

def get_rmses(models):
    rmses = []
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return rmses

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

y_target = house_df_ohe['SalePrice']
X_features = house_df_ohe.drop('SalePrice',axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_features,y_target, test_size=0.2, random_state =156)

lr_reg = LinearRegression()
lr_reg.fit(X_train,y_train)
ridge_reg = Ridge()
ridge_reg.fit(X_train,y_train)
lasso_reg = Lasso()
lasso_reg.fit(X_train,y_train)

models = [lr_reg,ridge_reg,lasso_reg]
get_rmses(models)

In [ ]:
def get_top_bottom(model,n=10):
    coef = pd.Series(model.coef_,index=X_features.columns)
    
    coef_high = coef.sort_values(ascending = False).head(n)
    coef_low = coef.sort_values(ascending=False).tail(n)
    return coef_high, coef_low

In [ ]:
def visulaize_coefficient(models):
    fig, axs = plt.subplots(figsize=(24,10),nrows=1,ncols=3)
    fig.tight_layout()
    for i_num,model in enumerate(models):
        coef_high, coef_low = get_top_bottom(model)
        coef_concat = pd.concat([coef_high,coef_low])
        axs[i_num].set_title(model.__class__.__name__+'Coefficients',size=25)
        axs[i_num].tick_params(axis='y',direction='in',pad=120)
        for label in (axs[i_num].get_xticklabels()+axs[i_num].get_yticklabels()):
            label.set_fontsize(22)
        sns.barplot(x=coef_concat.values,y=coef_concat.index,ax=axs[i_num])
        
models = [lr_reg,ridge_reg,lasso_reg]
visulaize_coefficient(models)

In [ ]:
from sklearn.model_selection import cross_val_score

def get_avg_rmse_cv(models):
    for model in models:
        rmse_list = np.sqrt(-cross_val_score(model,X_features,y_target,scoring='neg_mean_squared_error',cv=5))
        rmse_avg = np.mean(rmse_list)
        print('\n{0}CV RMSE 값 리스트: {1}'.format(model.__class__.__name__,np.round(rmse_list,3)))
        print('\n{0}CV RMSE 값 리스트: {1}'.format(model.__class__.__name__,np.round(rmse_avg,3)))
        
        
models = [lr_reg,ridge_reg, lasso_reg]
get_avg_rmse_cv(models)


In [ ]:
from sklearn.model_selection import GridSearchCV

def print_best_params(model,params):
    grid_model = GridSearchCV(model,param_grid=params,scoring='neg_mean_squared_error',cv=5)
    grid_model.fit(X_features,y_target)
    return print('{0} 5 CV시 최적 평균 RMSE 값:{1}, 최적 alpha: {2}'.format(model.__class__.__name__,np.round(get_rmse(model),3),grid_model.best_params_))
    
ridge_params = {'alpha':[0.05,0.1,1,5,8,10,12,15,20]}
lasso_params = {'alpha': [0.001,0.005,0.008,0.05,0.03,0.1,0.5,1,5,10]}
print_best_params(ridge_reg,ridge_params)
print_best_params(lasso_reg,lasso_params)

In [ ]:
lr_reg = LinearRegression()
lr_reg.fit(X_train,y_train)
ridge_reg = Ridge(alpha=12)
ridge_reg.fit(X_train,y_train)
lasso_reg = Lasso(alpha=0.01)
lasso_reg.fit(X_train,y_train)

models = [lr_reg,ridge_reg, lasso_reg]
visulaize_coefficient(models)

In [ ]:
from scipy.stats import skew

features_index = house_df.dtypes[house_df.dtypes != 'object'].index
skew_features = house_df[features_index].apply(lambda x: skew(x))

skew_features_top = skew_features[skew_features>1]
print(skew_features_top.sort_values(ascending=False))

In [ ]:
house_df[skew_features_top.index] = np.log1p(house_df[skew_features_top.index])

In [ ]:
plt.scatter(x=house_df_org['GrLivArea'],y=house_df_org['SalePrice'])
plt.ylabel('SalePrice',fontsize=15)
plt.xlabel('GrLivArea',fontsize=15)
plt.show()

In [ ]:
cond1 = house_df['GrLivArea'] > np.log1p(4000)
cond2 = house_df['SalePrice'] < np.log1p(500000)
outlier_index = house_df[cond1&cond2].index

print('Outlier Record index:',outlier_index.values)
print('Before outlier Elimination house_df_ohe shape:',house_df.shape)
house_df.drop(outlier_index,axis=0,inplace=True)
print('After outlier Elimination house_df_ohe shape:',house_df.shape)